In [1]:
import time
import qiskit
from qiskit import IBMQ, assemble, transpile
from qiskit.providers.jobstatus import JOB_FINAL_STATES
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise # import AER noise model
import numpy as np
import qiskit.ignis.verification.tomography as tom
from qiskit.providers.ibmq.managed import IBMQJobManager


In [10]:
# Uncomment below two lines on first use.
# IBMQ.enable_account("9dc5c937fcedf3575d64912c02715c0ec0e27043cf5f18040b0304cdd16dac2ea042ffecad4b007b245926a694f92a9ebfdc1ede6b381b004a2fe0ed2cf262b1")
# provider = IBMQ.load_account()
provider.backends()
# backend = provider.backends.ibmq_vigo



[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [4]:
qubit_list = [0,1,2,3,4]
q1 = QuantumRegister(len(qubit_list))
c1 = ClassicalRegister(len(qubit_list))

qc = QuantumCircuit(q1)
for j in range(5):
    qc.h(q1[j])
qc.cz(q1[0], q1[1])
qc.cz(q1[2], q1[3])
qc.cz(q1[0], q1[4])
qc.cz(q1[1], q1[2])
qc.cz(q1[3], q1[4])

print(qc)

bk_qasm = qiskit.Aer.get_backend('qasm_simulator')
bk_sv = qiskit.Aer.get_backend('statevector_simulator')
bkIBM_16 = provider.get_backend(name='ibmq_16_melbourne')
bkIBM_5 = provider.get_backend(name='ibmqx2')
bkIBM_qasm = provider.get_backend('ibmq_qasm_simulator')


      ┌───┐         
q0_0: ┤ H ├─■──■────
      ├───┤ │  │    
q0_1: ┤ H ├─■──┼──■─
      ├───┤    │  │ 
q0_2: ┤ H ├─■──┼──■─
      ├───┤ │  │    
q0_3: ┤ H ├─■──┼──■─
      ├───┤    │  │ 
q0_4: ┤ H ├────■──■─
      └───┘         


In [5]:
# state vector simulation
job = qiskit.execute(qc, bk_sv)
psi_qc = job.result().get_statevector(qc)

In [6]:
# qasm simulation
qctom = tom.state_tomography_circuits(qc, qubit_list, meas_labels='Pauli', meas_basis='Pauli')

# Execute the [[5,1,3]] state prep circuit (with the same noise model)
# job = qiskit.execute(ghztom, backend=backend, optimization_level=3, shots=1000, noise_model=noise_model)
job = qiskit.execute(qctom, backend=bk_qasm, optimization_level=3, shots=4096)
results = job.result()

tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# Perform the tomography fit
# which outputs a density matrix
rho_qc = tomo_qc.fit(method='lstsq')


# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


Fit Fidelity = 0.9926968757363109


In [11]:
# running on the machine
bcknd = bkIBM_16
max_job_count = 5
max_exp_per_job = 54
qctom = tom.state_tomography_circuits(qc, qubit_list, meas_labels='Pauli', meas_basis='Pauli')
# need to add measurements of syndromes and flags here
qctom = transpile(qctom, backend=bcknd)

job_manager = IBMQJobManager()
job_set_foo = job_manager.run(qctom, backend=bcknd, name='qctom', shots=2048, max_experiments_per_job=max_exp_per_job)

# single job
# qobj = assemble(, bkIBM_5, shots=2048)
# job = bkIBM_5.run(qobj)

start_time = time.time()
job_status = job_set_foo.statuses()
while job_status[max_job_count - 1] not in JOB_FINAL_STATES:
    print(f'Status @ {time.time()-start_time:0.0f} s: {job_status[max_job_count - 1]},'
          f' ')
    time.sleep(10)
    job_status = job_set_foo.statuses()
#     print(job_status)

results = job_set_foo.results()
# for _ in results:
#     print(results.get_counts(_))
    
tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# Perform the tomography fit
# which outputs a density matrix
rho_qc = tomo_qc.fit(method='lstsq')

# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 11 s: JobStatus.INITIALIZING, 
Status @ 24 s: JobStatus.QUEUED, 
Status @ 35 s: JobStatus.QUEUED, 
Status @ 47 s: JobStatus.QUEUED, 
Status @ 60 s: JobStatus.QUEUED, 
Status @ 71 s: JobStatus.QUEUED, 
Status @ 84 s: JobStatus.QUEUED, 
Status @ 95 s: JobStatus.QUEUED, 
Status @ 107 s: JobStatus.QUEUED, 
Status @ 119 s: JobStatus.QUEUED, 
Status @ 131 s: JobStatus.QUEUED, 
Status @ 143 s: JobStatus.QUEUED, 
Status @ 155 s: JobStatus.QUEUED, 
Status @ 167 s: JobStatus.QUEUED, 
Status @ 179 s: JobStatus.QUEUED, 
Status @ 191 s: JobStatus.QUEUED, 
Status @ 203 s: JobStatus.QUEUED, 
Status @ 215 s: JobStatus.QUEUED, 
Status @ 226 s: JobStatus.QUEUED, 
Status @ 238 s: JobStatus.QUEUED, 
Status @ 250 s: JobStatus.QUEUED, 
Status @ 262 s: JobStatus.QUEUED, 
Status @ 275 s: JobStatus.QUEUED, 
Status @ 287 s: JobStatus.QUEUED, 
Status @ 299 s: JobStatus.QUEUED, 
Status @ 310 s: JobStatus.QUEUED, 
Status @ 322 s: JobStatus.QUEUED, 
Status @ 334 s: Jo

In [12]:
# results = job_set_foo.results()
for i in range(3**(len(qubit_list))):
    print(results.get_counts(i))
print(i)
# print(results.get_counts(8))
# tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# # Perform the tomography fit
# # which outputs a density matrix
# rho_qc = tomo_qc.fit(method='lstsq')

{'11100': 34, '01111': 35, '01000': 52, '10011': 38, '00000': 90, '10110': 38, '00111': 38, '11010': 108, '01011': 77, '01101': 41, '00101': 50, '10001': 61, '00001': 110, '10100': 58, '11101': 57, '01110': 40, '10101': 76, '00010': 64, '11111': 56, '10111': 104, '00100': 80, '11011': 46, '11110': 37, '11000': 41, '01001': 65, '01100': 79, '00011': 66, '10000': 102, '11001': 51, '01010': 53, '00110': 97, '10010': 104}
{'11100': 61, '01111': 29, '01000': 38, '10011': 74, '00000': 76, '10110': 66, '00111': 49, '11010': 114, '01011': 99, '01101': 23, '00101': 54, '10001': 92, '00001': 87, '10100': 59, '11101': 72, '01110': 38, '10101': 63, '00010': 57, '11111': 60, '10111': 88, '00100': 74, '11011': 30, '11110': 49, '11000': 54, '01001': 86, '01100': 54, '00011': 32, '10000': 115, '11001': 29, '01010': 57, '00110': 83, '10010': 86}
{'11100': 66, '01111': 33, '01000': 38, '10011': 55, '00000': 80, '10110': 54, '00111': 54, '11010': 107, '01011': 82, '01101': 45, '00101': 54, '10001': 74, '

In [ ]:
# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


In [63]:
# result.data()
# result.get_memory()
# result.get_counts()
# result.get_statevector
# result.get_unitary
print(job_set_foo.statuses)
results = job_set_foo.results()


<bound method ManagedJobSet.statuses of <qiskit.providers.ibmq.managed.managedjobset.ManagedJobSet object at 0x000001E47F8CBFC8>>
